### 这个notebook用来测试addition evaluation

In [3]:
from main_utils import *

In [4]:
from model import GPTConfig, GPT
import torch

init_from = 'resume'


if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = 'bilabel_ckpt_acc.pt'
    checkpoint = torch.load(ckpt_path, map_location='cuda')
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)

number of parameters: 10.63M


In [5]:
# evaluation
config={
    'start': 'FILE:./data/addition_bilabel/prompt_3digit_10000.txt',
    'device': 'cuda',
}

In [7]:
encode, decode = get_encode_decode('./data/addition_bilabel/meta.pkl')

Loading meta from ./data/addition_bilabel/meta.pkl...


In [8]:
from contextlib import nullcontext

dtype = 'bfloat16'
device = 'cuda'
model = model.to(device)
device_type = 'cuda'
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]

ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
eval_addition_batch(config, model, ctx, encode, decode, judge=True)

evaluating addition from: FILE:./data/addition_bilabel/prompt_3digit_10000.txt


  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 81/81 [00:04<00:00, 16.58it/s]

Judgement accuracy of 10000 examples: 9350/10000 (93.5%)
accuracy of 10000 examples: 9350/10000 (93.5%)
{'carry0': 92.20543806646526, 'carry1': 90.75630252100841, 'carry2': 95.14619883040936, 'carry3': 98.1549815498155}


(93.5,
 93.5,
 {'carry0': 92.20543806646526,
  'carry1': 90.75630252100841,
  'carry2': 95.14619883040936,
  'carry3': 98.1549815498155})

In [167]:
x = '213+199=4121'
ids = encode(x)
input = (torch.tensor(ids, dtype=torch.long, device='cpu')[None, ...])
model.to(device='cpu')
output = model.generate(input, max_new_tokens=1)
decode(output[0].tolist())

'213+199=4121F'